https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html

In [ ]:
from keras.models import Sequential
from keras.optimizers import SGD
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import numpy as np
import os
from matplotlib.pyplot import imshow

In [ ]:
datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        horizontal_flip=True,
        fill_mode='wrap'
)


In [ ]:
img = load_img('data/test/nocrack/SampleA2_Detail_E_dog_101_206.png', grayscale=True)  # this is a PIL image
img

In [ ]:
x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
x.shape

In [ ]:
x/255

In [ ]:
#x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)
x = np.expand_dims(x, axis=0)
x.shape


In [ ]:
# the .flow() command below generates batches of randomly transformed images
# and saves the results to the `preview/` directory
#i = 0
#for batch in datagen.flow(x, batch_size=1, 
#                          save_to_dir='preview', save_prefix='img15', save_format='png'):
#    i += 1
#    if i > 10:
#        break  # otherwise the generator would loop indefinitely

In [ ]:
model = Sequential()
model.add(Conv2D(64, (3,3), input_shape=(100, 100, 1)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

In [ ]:
model.add(Conv2D(64, (3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

In [ ]:
model.add(Conv2D(64, (3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

In [ ]:
model.add(Conv2D(64, (3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

In [ ]:
model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.6))

In [ ]:
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])
model.summary()

In [ ]:
batch_size = 16

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rescale=1./255 ,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        #zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='wrap'
)


In [ ]:
test_datagen = ImageDataGenerator(
    rescale=1./255)


In [ ]:
train_generator = train_datagen.flow_from_directory(
        'data/train',  # this is the target directory
        target_size=(100, 100),  # all images will be resized to 150x150
        batch_size=batch_size,
        color_mode = 'grayscale',
        class_mode='binary')  # since we use binary_crossentropy loss, we need binary labels



In [ ]:
# this is a similar generator, for validation data
test_generator = test_datagen.flow_from_directory(
        'data/test',
        target_size=(100, 100),
        batch_size=batch_size,
        color_mode = 'grayscale',
        class_mode='binary')

In [ ]:
#model.load_weights('model_33.h5')
#model

In [ ]:
model.fit_generator(
        train_generator,
        steps_per_epoch=1678 // batch_size,
        epochs=70,
        validation_data=test_generator,
        validation_steps=1607 // batch_size,
        #initial_epoch= 70
)

In [ ]:
model.save_weights('model_33_weights.h5') 
model.save('model_33_model.h5') 

In [ ]:
from keras.models import load_model
from keras.preprocessing.image import img_to_array, load_img

test_model = load_model('model_33_model.h5')

for imgfile in os.listdir('data/test/crack'):
    print(imgfile)
    img = load_img('data/test/crack/' + imgfile,grayscale=True,target_size=(100,100))
    x = img_to_array(img)
    x = x/255
    x = np.expand_dims(x, axis=0)
    preds = test_model.predict_classes(x, verbose=0)
    probs = test_model.predict_proba(x, batch_size=1, verbose=0)
    print(preds, probs)

In [ ]:
for imgfile in os.listdir('data/test/nocrack'):
    print(imgfile)
    img = load_img('data/test/nocrack/' + imgfile,grayscale=True,target_size=(100,100))
    x = img_to_array(img)
    x = x/255
    x = np.expand_dims(x, axis=0)
    preds = test_model.predict_classes(x, verbose=0)
    probs = test_model.predict_proba(x, batch_size=1,verbose=0)
    print(preds, probs)